In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

url = "https://www.amazon.in/s?k=kettle&page={}&crid=2M096C61O4MLT&qid=1685083765&sprefix=ba%2Caps%2C283&ref=sr_pg_2"
num_pages = 20


rows = []
for page in range(1, num_pages + 1):
    page_url = url.format(page)
    retry_count = 0
    while retry_count < 100:  
        try:
            response = requests.get(page_url)
            if response.status_code == 503:
                #print("trying", page)
                time.sleep(5)  
                retry_count += 1
                continue

            response.raise_for_status()  
            time.sleep(3)  

            soup = BeautifulSoup(response.content, "html.parser")
            products = soup.find_all("div", class_="sg-col-inner")

            for product in products:
                new_row = []  

                url_element = product.find("a", class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")
                if url_element:
                    product_url = "https://www.amazon.in" + url_element["href"]
                    new_row.append(product_url)

                    name_element = product.find("span", class_="a-size-medium a-color-base a-text-normal")
                    if name_element:
                        product_name = name_element.text.strip()
                        new_row.append(product_name)
                    else:
                        new_row.append("")  

                    price_element = product.find("span", class_="a-price-whole")
                    if price_element:
                        product_price = price_element.text.strip()
                        new_row.append(product_price)
                    else:
                        new_row.append("") 

                    rating_element = product.find("span", class_="a-icon-alt")
                    if rating_element:
                        product_rating = rating_element.text.strip()
                        new_row.append(product_rating)
                    else:
                        new_row.append("")  

                    reviews_element = product.find("span", class_="a-size-base s-underline-text")
                    if reviews_element:
                        product_reviews_count = reviews_element.text.strip()
                        new_row.append(product_reviews_count)
                    else:
                        new_row.append("")  

                    rows.append(new_row)  
            break  
        except requests.exceptions.HTTPError as err:
            print("HTTP error occurred:", err)
            retry_count += 1  
        except requests.exceptions.RequestException as err:
            print("An error occurred:", err)
            retry_count += 1  

df = pd.DataFrame(rows, columns=['Product URL', 'Product Name', 'Product Price', 'Rating','Number of reviews'])
df = pd.DataFrame(rows, columns=['Product URL', 'Product Name', 'Product Price', 'Rating', 'Number of Reviews'])
df = df.drop_duplicates(subset=['Product URL','Product Name', 'Product Price', 'Rating', 'Number of Reviews'])
df = df.reset_index(drop=True)
#df.to_csv("kettle_price.csv")


In [91]:
df = pd.read_csv("/Users/apple/Desktop/kettle_price.csv", sep=',')
df

,Unnamed: 0,Product URL,Product Name,Product Price,Rating,Number of Reviews
0,0,https://www.amazon.in/Pigeon-Electric-Stainles...,"Pigeon Amaze Plus Electric Kettle 1.5 L, 1500 ...",599,3.9 out of 5 stars,"149,546"
1,1,https://www.amazon.in/Prestige-Electric-Kettle...,Prestige 1.5 Litres Electric Kettle (PKOSS 1.5...,699,3.9 out of 5 stars,"68,502"
2,2,https://www.amazon.in/Pigeon-Multipurpose-1217...,"Pigeon Amaze Multipurpose Kettle 600 Watt, 1.2...",899,3.8 out of 5 stars,"17,616"
3,3,https://www.amazon.in/Amazon-Basics-Electric-K...,NaN,599,4.0 out of 5 stars,739
4,4,https://www.amazon.in/Amazon-Basics-Electric-K...,NaN,599,4.0 out of 5 stars,379
...,...,...,...,...,...,...
313,313,https://www.amazon.in/Supvox%C2%AE-Electric-Po...,Supvox® 0.5 Litre Electric Kettle 300W Portabl...,"2,698",NaN,NaN
314,314,https://www.amazon.in/VYATIRANG-Stainless-Auto...,VYATIRANG Electric Kettle with Handle & Lid Do...,999,NaN,NaN
315,315,https://www.amazon.in/AxegooElectric-Kettle-St...,"AxegooElectric Kettle Hot Water Kettle, 1.8L S...",369,3.9 out of 5 stars,4
316,316,https://www.amazon.in/AxegooElectric-Kettle-St...,"AxegooElectric Kettle Hot Water Kettle, 1.8L S...",369,NaN,NaN


In [92]:
df['Rating'] = df['Rating'].str.replace(' out of 5 stars', '').astype(float)
df['Product Price'] = df['Product Price'].str.replace(',', '').astype(float)
df['Number of Reviews'] = df['Number of Reviews'].str.replace(',', '').astype(float)

In [93]:
df

,Unnamed: 0,Product URL,Product Name,Product Price,Rating,Number of Reviews
0,0,https://www.amazon.in/Pigeon-Electric-Stainles...,"Pigeon Amaze Plus Electric Kettle 1.5 L, 1500 ...",599.0,3.9,149546.0
1,1,https://www.amazon.in/Prestige-Electric-Kettle...,Prestige 1.5 Litres Electric Kettle (PKOSS 1.5...,699.0,3.9,68502.0
2,2,https://www.amazon.in/Pigeon-Multipurpose-1217...,"Pigeon Amaze Multipurpose Kettle 600 Watt, 1.2...",899.0,3.8,17616.0
3,3,https://www.amazon.in/Amazon-Basics-Electric-K...,NaN,599.0,4.0,739.0
4,4,https://www.amazon.in/Amazon-Basics-Electric-K...,NaN,599.0,4.0,379.0
...,...,...,...,...,...,...
313,313,https://www.amazon.in/Supvox%C2%AE-Electric-Po...,Supvox® 0.5 Litre Electric Kettle 300W Portabl...,2698.0,NaN,NaN
314,314,https://www.amazon.in/VYATIRANG-Stainless-Auto...,VYATIRANG Electric Kettle with Handle & Lid Do...,999.0,NaN,NaN
315,315,https://www.amazon.in/AxegooElectric-Kettle-St...,"AxegooElectric Kettle Hot Water Kettle, 1.8L S...",369.0,3.9,4.0
316,316,https://www.amazon.in/AxegooElectric-Kettle-St...,"AxegooElectric Kettle Hot Water Kettle, 1.8L S...",369.0,NaN,NaN


In [94]:
df_filtered = df_filtered.sort_values(by='Number of Reviews', ascending=True)

In [109]:
df_filtered = df[df['Product Price'] <= 2000]
df_filtered = df_filtered[df_filtered['Number of Reviews'] <= 50000]

In [110]:
df_filtered

,Unnamed: 0,Product URL,Product Name,Product Price,Rating,Number of Reviews
2,2,https://www.amazon.in/Pigeon-Multipurpose-1217...,"Pigeon Amaze Multipurpose Kettle 600 Watt, 1.2...",899.0,3.8,17616.0
3,3,https://www.amazon.in/Amazon-Basics-Electric-K...,NaN,599.0,4.0,739.0
4,4,https://www.amazon.in/Amazon-Basics-Electric-K...,NaN,599.0,4.0,379.0
5,5,https://www.amazon.in/AmazonBasics-Double-Wall...,NaN,1573.0,4.4,1441.0
6,6,https://www.amazon.in/AmazonBasics-Stainless-S...,NaN,996.0,4.2,2012.0
...,...,...,...,...,...,...
294,294,https://www.amazon.in/Guance-Electric-Stainles...,Guance 12V CAR Electric Stainless Steel Smart ...,800.0,2.1,3.0
295,295,https://www.amazon.in/Global-Litre-Stainless-S...,Kitchen Expert 5 Litre Stainless Steel Milk ca...,999.0,3.8,29.0
304,304,https://www.amazon.in/Prestige-Electric-Kettle...,"Prestige Electric Kettle PKOSS - 1500watts, St...",1398.0,4.6,4.0
315,315,https://www.amazon.in/AxegooElectric-Kettle-St...,"AxegooElectric Kettle Hot Water Kettle, 1.8L S...",369.0,3.9,4.0


In [118]:
import plotly.express as px

# Assuming you have a DataFrame called df_filtered with appropriate columns
fig = px.scatter(df_filtered, x="Product Price", y="Rating", size="Number of Reviews",
                 labels={"Product Price": "Price", "Rating": "Customer Rating"}, hover_data= ['Product Name'])

fig.show()